# Deep Hedging in Incomplete Markets — GBM & Heston

**MSc Thesis Experiment Runner**

Runs the full deep hedging pipeline under two market dynamics:
- **GBM** (constant volatility, calibrated to S&P 500)
- **Heston** (stochastic volatility, calibrated to S&P 500)

Before running:
1. **Runtime → Change runtime type → A100 GPU** (Pro+ recommended)
2. Click **Connect**

In [24]:
# Cell 1: Clone repo and install dependencies
!git clone https://github.com/thabangTheActuaryCoder/deep-hedging-thesis.git
%cd deep-hedging-thesis
!pip install -q torch numpy matplotlib optuna

import torch
print(f"\nPython: {__import__('sys').version}")
print(f"PyTorch: {torch.__version__}")
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")
    mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"Memory: {mem:.1f} GB")

Cloning into 'deep-hedging-thesis'...
remote: Enumerating objects: 183, done.
remote: Counting objects: 100% (183/183), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 183 (delta 74), reused 174 (delta 65), pack-reused 0 (from 0)
Receiving objects: 100% (183/183), 4.06 MiB | 3.61 MiB/s, done.
Resolving deltas: 100% (74/74), done.
/content/deep-hedging-thesis/deep-hedging-thesis/deep-hedging-thesis/deep-hedging-thesis/deep-hedging-thesis/deep-hedging-thesis

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch: 2.9.0+cu126
GPU available: True
Device: NVIDIA A100-SXM4-40GB
Memory: 42.5 GB


In [25]:
# Cell 2: Sanity check — all tests should pass
!python -m pytest tests/test_validation.py -v

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/deep-hedging-thesis/deep-hedging-thesis/deep-hedging-thesis/deep-hedging-thesis/deep-hedging-thesis/deep-hedging-thesis
plugins: anyio-4.12.1, typeguard-4.4.4, langsmith-0.6.8
collected 13 items                                                             

tests/test_validation.py::TestNoLookAhead::test_base_features_no_lookahead PASSED [  7%]
tests/test_validation.py::TestNoLookAhead::test_signature_features_no_lookahead PASSED [ 15%]
tests/test_validation.py::TestNoLookAhead::test_signature_at_zero PASSED [ 23%]
tests/test_validation.py::TestSelfFinancing::test_portfolio_update PASSED [ 30%]
tests/test_validation.py::TestSelfFinancing::test_path_terminal_consistency PASSED [ 38%]
tests/test_validation.py::TestSelfFinancing::test_no_future_prices_in_delta PASSED [ 46%]
tests/test_vali

In [ ]:
# Cell 3 (QUICK TEST): ~10 min on A100, verifies both GBM + Heston pipelines
!python run_experiment.py --quick --market_model both

Device: cuda
Quick mode: True
Market model: both
Paths=5000  Split hash=b954183058e99f18
Train=3000  Val=1000  Test=1000

  MARKET MODEL: GBM (calibrated)
  GBM: r=0.043, vols=[0.18, 0.22], extra_vol=0.06

  Pipeline: GBM market  (m_brownian=3)

=== [gbm] Step 2: Feature Construction ===
  Feature dim: 24

=== [gbm] Step 3: Stage 1 – Optuna HP Search (TPE) ===

--- FNN ---

  Optuna search for FNN: up to 3 trials (search space = 48 configs)


In [ ]:
# Cell 4 (FULL RUN): both GBM + Heston, 100k paths, ~4-8 hours on A100
# Comment out Cell 3 above before running this
!python run_experiment.py \
    --paths 100000 \
    --N 200 \
    --epochs 1000 \
    --patience 15 \
    --batch_size 2048 \
    --n_trials 60 \
    --seeds 0 1 2 3 4 \
    --substeps 0 5 10 \
    --market_model both

In [ ]:
# Cell 5: Preview GBM validation plots
from IPython.display import Image, display
import glob

print("=== GBM Validation Plots ===")
for img in sorted(glob.glob("outputs/gbm/plots_val/*.png")):
    print(f"\n--- {img} ---")
    display(Image(filename=img, width=700))

In [ ]:
# Cell 6: Preview Heston validation plots
from IPython.display import Image, display
import glob

print("=== Heston Validation Plots ===")
for img in sorted(glob.glob("outputs/heston/plots_val/*.png")):
    print(f"\n--- {img} ---")
    display(Image(filename=img, width=700))

In [ ]:
# Cell 7: Heston stochastic volatility diagnostic plots
from IPython.display import Image, display
import glob

print("=== Heston Stochastic Volatility Diagnostics ===")
for img in sorted(glob.glob("outputs/heston/plots_heston/*.png")):
    print(f"\n--- {img} ---")
    display(Image(filename=img, width=700))

In [ ]:
# Cell 8: GBM vs Heston comparison plots
from IPython.display import Image, display
import glob

print("=== GBM vs Heston Comparison ===")
for img in sorted(glob.glob("outputs/comparison/*.png")):
    print(f"\n--- {img} ---")
    display(Image(filename=img, width=700))

In [ ]:
# Cell 9: 3D delta surface plots
from IPython.display import Image, display
import glob

for label, pattern in [("GBM", "outputs/gbm/plots_3d/*.png"),
                       ("Heston", "outputs/heston/plots_3d/*.png")]:
    imgs = sorted(glob.glob(pattern))
    if imgs:
        print(f"\n=== {label} 3D Delta Surfaces ===")
        for img in imgs:
            print(f"\n--- {img} ---")
            display(Image(filename=img, width=700))

In [ ]:
# Cell 10: Show validation metrics (both market models)
import json, os

for market in ["gbm", "heston"]:
    path = f"outputs/{market}/val_metrics.json"
    if not os.path.exists(path):
        continue
    with open(path) as f:
        metrics = json.load(f)
    print(f"\n{'='*50}")
    print(f"  {market.upper()} — Best model: {metrics['best_model']}")
    print(f"{'='*50}")
    for model, agg in metrics["aggregated_val_metrics"].items():
        cvar = agg["CVaR95_shortfall"]
        mse = agg["MSE"]
        print(f"  {model:6s}  CVaR95 = {cvar['mean']:.6f} +/- {cvar['std']:.6f}  "
              f"MSE = {mse['mean']:.6f} +/- {mse['std']:.6f}")

# Combined summary
summary_path = "outputs/metrics_summary.json"
if os.path.exists(summary_path):
    with open(summary_path) as f:
        combined = json.load(f)
    print(f"\n{'='*50}")
    print("  COMBINED SUMMARY")
    print(f"{'='*50}")
    for market, agg in combined.items():
        print(f"\n  [{market.upper()}]")
        for model, m in agg.items():
            cvar = m.get("CVaR95_shortfall", {})
            if isinstance(cvar, dict):
                print(f"    {model:6s}  CVaR95 = {cvar.get('mean',0):.6f} +/- {cvar.get('std',0):.6f}")

## Replot

To adjust figure dimensions, colors, grids, or fonts without
rerunning the experiment, edit the `STYLE` dict in `replot.py` and rerun
Cell 12 below. The data was saved during the experiment.

In [ ]:
# Cell 12: Regenerate comparison plots from saved data (edit STYLE in replot.py first)
!python replot.py --data outputs/comparison/comparison_data.pt \
                  --metrics outputs/metrics_summary.json \
                  --out outputs/comparison

# Show regenerated plots
from IPython.display import Image, display
import glob
for img in sorted(glob.glob("outputs/comparison/*.png")):
    print(f"\n--- {img} ---")
    display(Image(filename=img, width=700))

In [ ]:
# Cell 13: Download all outputs as zip
import shutil
from google.colab import files

shutil.make_archive("outputs", "zip", ".", "outputs")
files.download("outputs.zip")